In [ ]:
# Imports 
import os
import numpy as np
import pandas as pd
pd.options.display.max_columns = None
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
from datetime import datetime, timedelta
#plt.style.use('ggplot')

# Mandatory install for xlsx files
# pip3 install xlrd

In [ ]:
# Global variables
EXCEL_FILEPATH = None
DICT_INFO_BY_ALUMN = {}
PDF_PATH = os.getcwd() + "\\pdf_by_alum\\" 
UNIQUE_NAMES = None  # Unique names of alumns
HEADS_FOLDER = os.getcwd() + "\\heads\\" 

In [ ]:
# Create pdf folder
def create_folder_from_path_if_not_exist(path):
    if not os.path.exists(PDF_PATH):
        os.makedirs(PDF_PATH)
        print("Directory " , PDF_PATH ,  " Created ")
    else:    
        print("Directory " , PDF_PATH ,  " already exists")

create_folder_from_path_if_not_exist(path=PDF_PATH)

In [ ]:
def get_last_daily_check():
    """ List all files in the same folder """
    for filepath in os.listdir('.'):
        file_name, extension = os.path.splitext(filepath)
        if "xlsx" in extension:
            return filepath

EXCEL_FILEPATH = os.getcwd() + "\\" + get_last_daily_check()
EXCEL_FILEPATH         

In [ ]:
df = pd.read_excel(EXCEL_FILEPATH)
df

In [ ]:
""" Columns to focus on:

- Fecha
- Nombre
- ¿Qué programa estás cursando?
- ¿Cómo te sientes al finalizar la jornada de clase? Puedes añadir tus propias opciones, si lo deseas.
- ¿Qué has aprendido hoy en clase?
- ¿Qué puntuación le darías a este día?

Renames:

- Date
- Name
- Bootcamp
- Sentiment
- Learnt
- Day_Score

"""

new_columns = ["Date", "Name", "Bootcamp", "Sentiment", "Learnt", "Day_Score"]

renames = {'Fecha':'Date',
           'Nombre':'Name',
           '¿Qué programa estás cursando?': 'Bootcamp',
           '¿Cómo te sientes al finalizar la jornada de clase? Puedes añadir tus propias opciones, si lo deseas.':'Sentiment',
           '¿Qué has aprendido hoy en clase? ':'Learnt',
           '¿Qué puntuación le darías a este día?':'Day_Score',
            }

# Rename columns
df.rename(columns=renames, inplace=True)    

In [ ]:
def sentiment_transformation(x):
    value = x.lower()
    if "contento" in value:
        return 5
    elif "triste" in value:
        return 4
    elif "enfadado" in value:
        return 3
    elif "asustado" in value:
        return 2
    else:
        return 1
    
df["Sentiment"] = df["Sentiment"].apply(sentiment_transformation)
df

In [ ]:
# df index
df.set_index("Date", inplace=True)

In [ ]:
UNIQUE_NAMES = df.Name.unique()

# Create dict with the values of Learnt column by alumn
for name in UNIQUE_NAMES:
    # [Bootcamp, Sentiment, Day_Score, Learnt]
    alumn_info = []
    # Bootcamp
    alumn_info.append(max(set(list(df[df.Name == name]["Bootcamp"].values)), key = list(df[df.Name == name]["Bootcamp"].values).count))
    # Sentiment
    alumn_info.append(df[df.Name == name]["Sentiment"])
    # Day_Score
    alumn_info.append(df[df.Name == name]["Day_Score"])
    # Learnt
    alumn_info.append(df[df.Name == name]["Learnt"])
    DICT_INFO_BY_ALUMN[name] = alumn_info

In [ ]:
"""
# Delete unnecesary columns
columns_to_delete = [column for column in df.columns if column not in new_columns]  
columns_to_delete.append("Learnt")
df.drop(columns_to_delete, axis=1, inplace=True)
"""
df



### DICT_INFO_BY_ALUMN

 {**alumn**: [[Bootcamp,
               Sentiment, 
               Day_Score, 
               Learnt]]}



## TO PDF 

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox

def getImage(path):
    return OffsetImage(plt.imread(path))

def choose_image(x, y, ax):
    for x0, y0 in zip(x, y):
        if y0 > 4:
            path = HEADS_FOLDER + "5_smile.png"
        elif y0 == 4:
            path = HEADS_FOLDER + "4_sad.png"
        elif y0 == 3:
            path = HEADS_FOLDER + "3_angry.png"
        elif y0 == 2:
            path = HEADS_FOLDER + "2_scary.png"
        else:
            path = HEADS_FOLDER + "1_confused.png"

        ab = AnnotationBbox(getImage(path), (x0, y0), pad=0, frameon=False)
        ax.add_artist(ab)

paths = [
    HEADS_FOLDER + "1_confused.png",
    HEADS_FOLDER + "2_scary.png",
    HEADS_FOLDER + "3_angry.png",
    HEADS_FOLDER + "4_sad.png",
    HEADS_FOLDER + "5_smile.png"]

In [ ]:
# constants
dpi = 72
imageSize = (32,32)
# read in our png file
one_confused = image.imread(HEADS_FOLDER + "1_confused.png")
two_scary = image.imread(HEADS_FOLDER + "2_scary.png")
three_angry = image.imread(HEADS_FOLDER + "3_angry.png")
four_sad = image.imread(HEADS_FOLDER + "4_sad.png")
five_happy = image.imread(HEADS_FOLDER + "5_smile.png")

def reformat_string_if_longer_than_n_chars(string, n):

    if len(string) > n:
        split_strings = [string[index : index + n] for index in range(0, len(string), n)]
        last_string = split_strings[-1]
        string = "".join((s + "\n") for s in split_strings[:-2])
        string += last_string
    return string

with PdfPages(PDF_PATH + "1_Global_Analysis.pdf") as global_pdf:
    print("Analyzing...")
    for alumn_name in UNIQUE_NAMES:
        alumn_bootcamp = DICT_INFO_BY_ALUMN[alumn_name][0]
        txt = "What have you learnt?\n\n"
        title = (alumn_name + " | " + alumn_bootcamp) if len((alumn_name + " | " + alumn_bootcamp)) <=54 else (alumn_name + "\n" + alumn_bootcamp)
        pdf_name = alumn_bootcamp + "_" + alumn_name + ".pdf"
        #create_folder_from_path_if_not_exist(path=PDF_PATH)
        pdf_name = pdf_name.replace("/","-")
        with PdfPages(PDF_PATH + pdf_name) as alumn_pdf:
            
            """ First Page """
            fig = plt.figure(figsize=(8.69,11.27), frameon=False)

            ax = fig.add_axes([0, 0, 1, 1])
            ax.axis('off')
            #plt.title(alumn_name + " | " + alumn_bootcamp)
            txt += "".join(("- " + reformat_string_if_longer_than_n_chars(string, 70) + "\n") for string in DICT_INFO_BY_ALUMN[alumn_name][3])
            fig.text(0.5, 0.9, title, ha='center', size=20, family='serif' )
            fig.text(0.1, 0.6, txt, ha='left', horizontalalignment='left', verticalalignment='center', bbox=dict(facecolor='grey', alpha=0.2), size=13)
            plt.title(title)
            alumn_pdf.savefig(orientation='portrait', dpi=600)

            """ Second Page """

            # Sentiment plot 
            fig2, (ax1, ax2) = plt.subplots(2, figsize=(8.69,11.27), constrained_layout=True)
            #fig.subplots_adjust(bottom=1.4, top=1.5)
            fig2.subplots_adjust(hspace=-.5)
            fig2.tight_layout()
            fig2.autofmt_xdate()

            plt.suptitle("Sentiment & Day Score plots", x=0.5, y=.1, horizontalalignment='center', verticalalignment='top', fontsize = 22)
    
            index = DICT_INFO_BY_ALUMN[alumn_name][1].index
            sentiment_data = DICT_INFO_BY_ALUMN[alumn_name][1].values

            #ax1.legend()
            #ax1.spines["left"].set_position(("axes", .03))
            ax1.spines["bottom"].set_bounds(0, 0)
            ax1.set_facecolor("#fffffd")
            #ax1.spines["bottom"].set_linewidth(6)
            ax1.grid(True)
            ax1.scatter(index, sentiment_data, label="Sentiment")
            ax1.fmt_xdata = mdates.DateFormatter('%m-%d')
            choose_image(x=index, y=sentiment_data, ax=ax1)
            
            
            ax1.set_xlim([index[0] - timedelta(3), index[0] + timedelta(23)])
            ax1.set_ylim([0, 5])

            # Day_Score
            index = DICT_INFO_BY_ALUMN[alumn_name][2].index
            day_score = DICT_INFO_BY_ALUMN[alumn_name][2].values
            
            #ax2.legend()
            ax2.spines["top"].set_bounds(0, 0)
            ax2.set_facecolor("#fffffd")
            ax2.grid(True)
            ax2.scatter(index, day_score, label="Day Score")
            ax2.fmt_xdata = mdates.DateFormatter('%m-%d')
            choose_image(x=index, y=day_score, ax=ax2)

            ax2.set_xlim([index[0] - timedelta(3), index[0] + timedelta(23)])
            ax2.set_ylim([0, 6])

            alumn_pdf.savefig(orientation='portrait', dpi=600)

        global_pdf.savefig(orientation='portrait', dpi=600)  
        plt.close()
    print("Alumns analyzed!")